# JAX-Fluids: Numerical setup demo
This demo will show what a typical numerical setup file contains and how you can change the numerical setup for your simulation. We use the case setup "01_case_setup_sod.json" from an earlier tutorial and the numerical setup "01_numerical_setup_sod.json".

In [ ]:
import json
from jaxfluids import InputReader, Initializer, SimulationManager
from jaxfluids.post_process import load_data, create_lineplot

## Numerical Setup
In JAX-Fluids we typically specify the numerical setup via a json file. In the first tutorial we have seen that we can pass the path to any numerical setup as a string to the InputReader. Internally, the InputReader will read the json file and convert it to a dictionary.

In [ ]:
input_reader = InputReader("01_case_setup_sod.json", "01_numerical_setup_sod.json")

However, it is also possible to pass a dictionary directly to the InputReader. Let us load the 01_numerical_setup_sod.json file and look what it contains.

In [ ]:
numerical_setup = json.load(open("01_numerical_setup_sod.json"))
numerical_setup.keys()

The case setup specifies what numerical schemes are used for your simulation. Here, the numerical setup contains the keys "conservatives", "active_physics", "active_forcings", and "output". Let us have a look which numerical methods are used to integrate the buffer of conservative variables.

In [ ]:
numerical_setup["conservatives"]

Currently we use 4 halo cells for the boundary conditions and the time integrator is a TVD Runge-Kutta 3 scheme. The convective fluxes are calculated with a WENO5-JS reconstruction scheme and an HLLC Riemann solver. We exchange the HLLC Riemann solver for an HLL Riemann solver and use a WENO3-JS reconstruction scheme.

In [ ]:
numerical_setup["conservatives"]["convective_fluxes"]["riemann_solver"] = "HLL"
numerical_setup["conservatives"]["convective_fluxes"]["spatial_reconstructor"] = "WENO3-JS"

The updated numeric

In [ ]:
numerical_setup["conservatives"]

## Simulation Setup
To run a forward simulation with JAX-Fluids, you need an InputRead, an Initializer, and a SimulationManager.

The InputReader reads a case setup file and a numerical setup file. Now, we pass the updated case setup dictionary to the InputReader.

In [ ]:
input_reader = InputReader("01_case_setup_sod.json", numerical_setup)
initializer  = Initializer(input_reader)
sim_manager  = SimulationManager(input_reader)

## Run Simulation
The Initializer creates a dictionary which contains all initial flow fields. The SimulationManager runs the simulation and writes h5 output files.

In [ ]:
buffer_dictionary = initializer.initialization()
sim_manager.simulate(buffer_dictionary)

## Load Data
JAX-Fluids offers post-processing tools for visualizing your simulation data. You can find them under jaxfluids.post_process.

In [ ]:
path = sim_manager.output_writer.save_path_domain
quantities = ["density", "velocityX", "pressure"]
cell_centers, cell_sizes, times, data_dict = load_data(path, quantities)

## Visualize Data

In [ ]:
nrows_ncols = (1,3)
create_lineplot(data_dict, cell_centers, times, nrows_ncols=nrows_ncols, interval=100, static_time=0.2)